##Some preliminaries, you will need 
- package Match (Pkg.add("Match")) 
- PyCall
- a working Python, with scipy, numpy and qutip installed.

In [132]:
reload("PTSM.jl")
reload("Initial.jl")
reload("Symplectic.jl")

In [2]:
using PyCall

In [3]:
@pyimport scipy.optimize as so
@pyimport numpy as np
@pyimport matplotlib.pyplot as plt
@pyimport qutip as qt
@pyimport numpy.linalg as la

## We need to get the Python finite field package working

In [4]:
unshift!(PyVector(pyimport("sys")["path"]), "./finite-fields/")
@pyimport finitefield as ff

In [5]:
F22 = ff.FiniteField(2,2)

1 + 1 x^1 + 1 x^2


fn (generic function with 1 method)

In [6]:
@pyimport SLFunctions as sl

In [7]:
function getStabilisers()
    jstabs=Array{Int64,2}[]
    stabs = sl.getF2Stabilisers()
    for i in 1:60
        push!(jstabs,reshape(stabs[i,:,:],4,4))
    end
    return jstabs
end


getStabilisers (generic function with 1 method)

In [8]:
stabs=getStabilisers();

##So we have the F22 stabilisers (60 of them) loaded into stabs

#Optimise the gates required for these 60
##!WARNING
these take a long time to run! - The data is now saved, just load it, next worksheet.

In [9]:
#watch out currently fails for 29 (as its the identity)
#gatesNeeded=Array{Int32,1}[]
#for i=1:60
#    state=setup(2)
#    state[:,1:4]=stabs[i]
#    back=bruteForceBreadthFirst(state)
#    push!(gatesNeeded,back)
#    writedlm("gateList$i.csv",back)
#end
    

In [9]:
#since we saved them previously we can load them
gates=Array{Float64}[]
for i=1:60
    push!(gates,readdlm("compiledGates/gateList$i.csv"))
end

In [10]:
gates[29]=[7.0]

1-element Array{Float64,1}:
 7.0

##The saved gates are indexes to the following "gate" array

In [11]:
gatesCommand = (Expr)[]
	svec = []
	for i = 1:2
	  for j = 1:2
	  	if i!=j
            push!(gatesCommand,Expr(:call,:cnot,:svec,i,j,false))
	  	end
	  end
    push!(gatesCommand,Expr(:call,:hadamard,:svec,i,false))
    push!(gatesCommand,Expr(:call,:phase,:svec,i,false))
	end
push!(gatesCommand,Expr(:call,:noop))

7-element Array{Expr,1}:
 :(cnot(svec,1,2,false))  
 :(hadamard(svec,1,false))
 :(phase(svec,1,false))   
 :(cnot(svec,2,1,false))  
 :(hadamard(svec,2,false))
 :(phase(svec,2,false))   
 :(noop())                

## Or if we just want to print it

In [12]:
gatestring = ASCIIString[]
svec = []
for i = 1:2
  for j = 1:2
  	if i!=j
            push!(gatestring,"cnot($i,$j)")
  	end
  end
  push!(gatestring,"hadamard($i)")
  push!(gatestring,"phase($i)")
end
push!(gatestring,"Identity")

7-element Array{ASCIIString,1}:
 "cnot(1,2)"  
 "hadamard(1)"
 "phase(1)"   
 "cnot(2,1)"  
 "hadamard(2)"
 "phase(2)"   
 "Identity"   

In [13]:
function makeFromCommand(command)
    currentState = [1 0 0 0;0 1 0 0;0 0 1 0;0 0 0 1]
    si = [1 0;0 1]
    sphase1=kron([1 0;0 im],si)
    sphase2=kron(si,[1 0;0 im])
    shadmard=1/sqrt(2)*[1 1;1 -1]
    shad1 = kron(shadmard,si)
    shad2 = kron(si,shadmard)
    cnot12 = [1 0 0 0;0 1 0 0;0 0 0 1;0 0 1 0]
    cnot21 = [1 0 0 0;0 0 0 1;0 0 1 0;0 1 0 0]
    for t in command
        m = match(r"setup\((.*)\)",t)
        if (m!=nothing)
           state=[1 0 0 0;0 1 0 0;0 0 1 0;0 0 0 1]
        else 
           m=match(r"phase\((.*)\)",t)
           if (m!=nothing)
                bit = int(m.captures[1])
                if bit == 1
                   currentState=currentState*sphase1
                else 
                   currentState=currentState*sphase2
                end
            else
                m=match(r"hadamard\((.*)\)",t)
                if (m!=nothing)
                    bit = int(m.captures[1])
                    if bit == 1
                        currentState=currentState*shad1
                    else 
                        currentState=currentState*shad2
                    end
                else
                    m=match(r"cnot\((.*),(.*)\)",t)
                    if (m!=nothing)
                        cbit = int(m.captures[1])
                        tbit = int(m.captures[2])
                        if cbit ==1 
                            currentState = currentState*cnot12
                        else
                            currentState = currentState*cnot21
                        end
                    end
                end
            end
        end
    end
    return  currentState=currentState
end


makeFromCommand (generic function with 1 method)

In [ ]:
# if you haven't already you may need to: Pkg.add("ProgressMeter")
# or just remove the ProgressMeter references, they aren't actually needed.

In [14]:
using ProgressMeter

###Looks good, so lets make them all

In [15]:
sl2Cliffords= Array{Complex{Float64},2}[]
sl2CliffordCommands = Array{ASCIIString,1}[]
n=60*4*4
p=Progress(n,1)
for i=1:60
    for j=1:4
        for z=1:4
            state=setup(2)
            state[:,1:4]=stabs[i]
            decomposeState(state,true)
            #Apply the PAULIS!
            if (j==2)
                push!(commands,"hadamard(1)")
                push!(commands,"phase(1)")
                push!(commands,"phase(1)")
                push!(commands,"hadamard(1)")
            elseif (j==3)
                push!(commands,"phase(1)")
                push!(commands,"phase(1)")
            elseif (j==4)
                push!(commands,"phase(1)")
                push!(commands,"phase(1)")
                push!(commands,"hadamard(1)")
                push!(commands,"phase(1)")
                push!(commands,"phase(1)")
                push!(commands,"hadamard(1)")
            end
            if (z==2)
                push!(commands,"hadamard(2)")
                push!(commands,"phase(2)")
                push!(commands,"phase(2)")
                push!(commands,"hadamard(2)")
            elseif (z==3)
                push!(commands,"phase(2)")
                push!(commands,"phase(2)")
            elseif (z==4)
                push!(commands,"phase(2)")
                push!(commands,"phase(2)")
                push!(commands,"hadamard(2)")
                push!(commands,"phase(2)")
                push!(commands,"phase(2)")
                push!(commands,"hadamard(2)")
            end
            push!(sl2CliffordCommands,commands)
            push!(sl2Cliffords,makeFromCommand(commands))
            next!(p)
        end
    end
end

Progress: 100% Time: 0:00:08


In [16]:
size(sl2Cliffords)

(960,)

In [21]:
sl2CliffordCommands[1]

4-element Array{ASCIIString,1}:
 "setup(2)"    
 "hadamard(2)" 
 "hadamard(1)" 
 "output(svec)"

#so the number of gates is the length of the commands minus 2

In [23]:
sl2CliffordsLength = [size(sl2CliffordCommands[i],1)-2 for i=1:size(sl2CliffordCommands,1)]
print("Size is $(size(sl2CliffordsLength,1)) and mean is $(mean(sl2CliffordsLength))")

Size is 960 and mean is 17.2

##Check if it is unitary 2

In [17]:
sum = 0
numb = size(sl2Cliffords,1)
for i=1:numb
    for j=1:numb
        sum += abs(trace(conj(sl2Cliffords[i])'*sl2Cliffords[j]))^4
    end
end
print("$sum -> $(sum/numb^2))\n")

1.8431999999999998e6 -> 1.9999999999999998)


##And the magic number is indeed 2

## Now lets compare with the actual full Clifford Group.

In [18]:
getNumberOfCliffords(2)

11520

In [19]:
getNumberOfSymplecticCliffords(2) # this is the number mapped to integers

720

In [20]:
getNumberOfBitStringsCliffords(2) # this is the number we multiply it by

16

In [134]:
FullCliffords= Array{Complex{Float64},2}[]
FullCliffordCommands = Array{ASCIIString,1}[]
n=11520
p=Progress(n,1)
for i=0:719 # number of symplectics
    for j=0:15 # number of phase variations.
        state=setup(2)
        decompose(i,j,2,true,false) # symplectic i, bits j, 2 qubits, no output and rationalise
        push!(FullCliffordCommands,commands)
        push!(FullCliffords,makeFromCommand(commands))
        next!(p)
    end
end

Progress: 100% Time: 0:00:06


In [125]:
for i=0:15
    decompose(0,i,2,false,false)
end

Tableau for unitary: 
+XI
+IX
---
+ZI
+IZ
setup(2)
output(svec)
Tableau for unitary: 
-XI
+IX
---
+ZI
+IZ
setup(2)
phase(1)
phase(1)
output(svec)
Tableau for unitary: 
+XI
-IX
---
-ZI
+IZ
setup(2)
phase(2)
phase(2)
hadamard(2)
hadamard(1)
phase(1)
phase(1)
hadamard(2)
hadamard(1)
output(svec)
Tableau for unitary: 
-XI
-IX
---
-ZI
+IZ
setup(2)
phase(2)
phase(2)
phase(1)
phase(1)
hadamard(2)
hadamard(1)
phase(1)
phase(1)
hadamard(2)
hadamard(1)
output(svec)
Tableau for unitary: 
+XI
+IX
---
+ZI
-IZ
setup(2)
hadamard(2)
hadamard(1)
phase(2)
phase(2)
hadamard(2)
hadamard(1)
output(svec)
Tableau for unitary: 
-XI
+IX
---
+ZI
-IZ
setup(2)
phase(1)
phase(1)
hadamard(2)
hadamard(1)
phase(2)
phase(2)
hadamard(2)
hadamard(1)
output(svec)
Tableau for unitary: 
+XI
-IX
---
-ZI
-IZ
setup(2)
phase(2)
phase(2)
hadamard(2)
hadamard(1)
phase(2)
phase(2)
phase(1)
phase(1)
hadamard(2)
hadamard(1)
output(svec)
Tableau for unitary: 
-XI
-IX
---
-ZI
-IZ
setup(2)
phase(2)
phase(2)
phase(1)
phase(1)
hadamard(

In [124]:
for j=1:16
    print(j,"\n")
end

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16


In [131]:
n=2
for bits=1:16
    top = [ (bits >> (x-1)) %2 for x=1:n]
    for co=1:n
        bits = div(bits,2)
    end
    bottom = [ (bits >> (x-1)) %2 for x=1:n]
    print(bits,"--->",top," ",bottom,"\n")
end

0--->{1,0} {0,0}
0--->{0,1} {0,0}
0--->{1,1} {0,0}
1--->{0,0} {1,0}
1--->{1,0} {1,0}
1--->{0,1} {1,0}
1--->{1,1} {1,0}
2--->{0,0} {0,1}
2--->{1,0} {0,1}
2--->{0,1} {0,1}
2--->{1,1} {0,1}
3--->{0,0} {1,1}
3--->{1,0} {1,1}
3--->{0,1} {1,1}
3--->{1,1} {1,1}
4--->{0,0} {0,0}


In [128]:
n=2
bits=1
top = [ (bits >> (x-1)) %2 for x=1:n]
bits = div(bits,2)
bottom = [ (bits >> (x-1)) %2 for x=1:n]
print(top," ",bottom)

2-element Array{Any,1}:
 0
 0

In [129]:
top

2-element Array{Any,1}:
 1
 0

In [22]:
size(FullCliffords)

(11520,)

In [23]:
FullCliffords[1]

4x4 Array{Complex{Float64},2}:
  0.707107+0.0im        0.0+0.0im  0.707107+0.0im       0.0+0.0im
       0.0+0.0im   0.707107+0.0im       0.0+0.0im  0.707107+0.0im
 -0.707107+0.0im        0.0+0.0im  0.707107+0.0im       0.0+0.0im
       0.0+0.0im  -0.707107+0.0im       0.0+0.0im  0.707107+0.0im

## Check we have done it correct (unitary 2)

In [135]:
sum = 0
numb = size(FullCliffords,1)
p=Progress(numb,1)
for i=1:numb
    for j=1:numb
        sum += abs(trace(conj(FullCliffords[i])'*FullCliffords[j]))^4
    end
    next!(p)
end
print("$sum -> $(sum/numb^2))\n")

Progress: 100% Time: 0:06:11
2.6542079999999997e8 -> 1.9999999999999998)


##Hmm probably a rounding error.

## So recap
- we have in sl2Cliffords each of the 4 x 4 matrices that form our smaller unitary 2 design (960)
- we have in FullCLiffords each of the 4 x 4 matrices taht form our full clifford group (11520)
- we are going to make a SuperOperator (Pauli Basis) for each of the full clifford group and store this
- we are going to store and INDEX for the sl2 cliffords to the SuperOperator index
- we are going to identify our generators, superoperator and find their indices.
- we are going to make a tree with 11520 vertices
- then we multiply each $CL_i^\{dagger},CL_j$ to see if they are a generator and set the edge appropriately 

In [25]:
#Granade inspired
function to_superpauli(S):
    """
    Converts a superoperator in the column-stacking basis to the Pauli basis. Note that
    the argument is assumed to act on qubits.
    """
    sqobj=qt.to_super(S)
    nq=int(np.log2(sqobj[:shape][1])/2)
    B=pycall(qt.superop_reps["_pauli_basis"],PyAny,nq)
    sq=np.sqrt(2^nq)
    B=pyeval("B/sq",PyAny,B=B,nq=nq,sq=sq)
    B[:dims] = sqobj[:dims]
    sqobj = pycall(B[:dag]()["__mul__"],PyAny,pycall(sqobj["__mul__"],PyAny,B))
    return sqobj
end

to_superpauli (generic function with 1 method)

In [48]:
SuperCliffs = [int(real(to_superpauli(qt.Qobj(FullCliffords[i]))[:full]())) for i =1:length(FullCliffords)]

11520-element Array{Any,1}:
 16x16 Array{Int64,2}:
 1  0  0  0  0  0  0  0  0  0  0  0   0   0   0   0
 0  1  0  0  0  0  0  0  0  0  0  0   0   0   0   0
 0  0  1  0  0  0  0  0  0  0  0  0   0   0   0   0
 0  0  0  1  0  0  0  0  0  0  0  0   0   0   0   0
 0  0  0  0  0  0  0  0  0  0  0  0  -1   0   0   0
 0  0  0  0  0  0  0  0  0  0  0  0   0  -1   0   0
 0  0  0  0  0  0  0  0  0  0  0  0   0   0  -1   0
 0  0  0  0  0  0  0  0  0  0  0  0   0   0   0  -1
 0  0  0  0  0  0  0  0  1  0  0  0   0   0   0   0
 0  0  0  0  0  0  0  0  0  1  0  0   0   0   0   0
 0  0  0  0  0  0  0  0  0  0  1  0   0   0   0   0
 0  0  0  0  0  0  0  0  0  0  0  1   0   0   0   0
 0  0  0  0  1  0  0  0  0  0  0  0   0   0   0   0
 0  0  0  0  0  1  0  0  0  0  0  0   0   0   0   0
 0  0  0  0  0  0  1  0  0  0  0  0   0   0   0   0
 0  0  0  0  0  0  0  1  0  0  0  0   0   0   0   0                                                                
 16x16 Array{Int64,2}:
 1   0   0  0   0  0  0   0  0

In [45]:
int(real(ar))

16x16 Array{Int64,2}:
 1  0  0  0   0   0   0   0  0  0  0  0  0  0  0  0
 0  1  0  0   0   0   0   0  0  0  0  0  0  0  0  0
 0  0  1  0   0   0   0   0  0  0  0  0  0  0  0  0
 0  0  0  1   0   0   0   0  0  0  0  0  0  0  0  0
 0  0  0  0   0   0   0   0  1  0  0  0  0  0  0  0
 0  0  0  0   0   0   0   0  0  1  0  0  0  0  0  0
 0  0  0  0   0   0   0   0  0  0  1  0  0  0  0  0
 0  0  0  0   0   0   0   0  0  0  0  1  0  0  0  0
 0  0  0  0  -1   0   0   0  0  0  0  0  0  0  0  0
 0  0  0  0   0  -1   0   0  0  0  0  0  0  0  0  0
 0  0  0  0   0   0  -1   0  0  0  0  0  0  0  0  0
 0  0  0  0   0   0   0  -1  0  0  0  0  0  0  0  0
 0  0  0  0   0   0   0   0  0  0  0  0  1  0  0  0
 0  0  0  0   0   0   0   0  0  0  0  0  0  1  0  0
 0  0  0  0   0   0   0   0  0  0  0  0  0  0  1  0
 0  0  0  0   0   0   0   0  0  0  0  0  0  0  0  1

In [49]:
test=SuperCliffs[6]


16x16 Array{Int64,2}:
 1   0  0   0   0  0   0  0  0   0  0   0  0   0  0   0
 0  -1  0   0   0  0   0  0  0   0  0   0  0   0  0   0
 0   0  1   0   0  0   0  0  0   0  0   0  0   0  0   0
 0   0  0  -1   0  0   0  0  0   0  0   0  0   0  0   0
 0   0  0   0   0  0   0  0  0   0  0   0  1   0  0   0
 0   0  0   0   0  0   0  0  0   0  0   0  0  -1  0   0
 0   0  0   0   0  0   0  0  0   0  0   0  0   0  1   0
 0   0  0   0   0  0   0  0  0   0  0   0  0   0  0  -1
 0   0  0   0   0  0   0  0  1   0  0   0  0   0  0   0
 0   0  0   0   0  0   0  0  0  -1  0   0  0   0  0   0
 0   0  0   0   0  0   0  0  0   0  1   0  0   0  0   0
 0   0  0   0   0  0   0  0  0   0  0  -1  0   0  0   0
 0   0  0   0  -1  0   0  0  0   0  0   0  0   0  0   0
 0   0  0   0   0  1   0  0  0   0  0   0  0   0  0   0
 0   0  0   0   0  0  -1  0  0   0  0   0  0   0  0   0
 0   0  0   0   0  0   0  1  0   0  0   0  0   0  0   0

In [50]:
findfirst(SuperCliffs,test)

6

In [51]:
sl2Cliffords[5]

4x4 Array{Complex{Float64},2}:
  0.5+0.0im   0.5+0.0im  0.5+0.0im   0.5+0.0im
  0.5+0.0im  -0.5+0.0im  0.5+0.0im  -0.5+0.0im
 -0.5+0.0im  -0.5+0.0im  0.5+0.0im   0.5+0.0im
 -0.5+0.0im   0.5+0.0im  0.5+0.0im  -0.5+0.0im

In [53]:
indexOfSl2 = [findfirst(SuperCliffs,int(real(to_superpauli(qt.Qobj(sl2Cliffords[i]))[:full]()))) for i=1:length(sl2Cliffords)]

960-element Array{Any,1}:
  1928
     0
  1924
     0
  1921
     0
  1925
     0
     0
  1922
     0
  1926
     0
     ⋮
     0
 10167
 10164
     0
     0
 10161
 10162
     0
 10166
     0
     0
 10165

In [55]:
findfirst(SuperCliffs,int(real(to_superpauli(qt.Qobj(sl2Cliffords[2]))[:full]())))

0

In [56]:
int(real(to_superpauli(qt.Qobj(sl2Cliffords[2]))[:full]()))

16x16 Array{Int64,2}:
 1  0  0   0  0  0  0   0   0   0   0  0  0  0  0   0
 0  0  0  -1  0  0  0   0   0   0   0  0  0  0  0   0
 0  0  1   0  0  0  0   0   0   0   0  0  0  0  0   0
 0  1  0   0  0  0  0   0   0   0   0  0  0  0  0   0
 0  0  0   0  0  0  0   0   0   0   0  0  1  0  0   0
 0  0  0   0  0  0  0   0   0   0   0  0  0  0  0  -1
 0  0  0   0  0  0  0   0   0   0   0  0  0  0  1   0
 0  0  0   0  0  0  0   0   0   0   0  0  0  1  0   0
 0  0  0   0  0  0  0   0  -1   0   0  0  0  0  0   0
 0  0  0   0  0  0  0   0   0   0   0  1  0  0  0   0
 0  0  0   0  0  0  0   0   0   0  -1  0  0  0  0   0
 0  0  0   0  0  0  0   0   0  -1   0  0  0  0  0   0
 0  0  0   0  1  0  0   0   0   0   0  0  0  0  0   0
 0  0  0   0  0  0  0  -1   0   0   0  0  0  0  0   0
 0  0  0   0  0  0  1   0   0   0   0  0  0  0  0   0
 0  0  0   0  0  1  0   0   0   0   0  0  0  0  0   0

In [57]:
factorial(16)

20922789888000

In [65]:
findfirst(SuperCliffs,int(real(to_superpauli(qt.Qobj(kron([0 1;1 0],[1 0;0 1])))[:full]())))

0

In [69]:
findfirst(FullCliffords,kron([0im 1;1 0],[1 0;0 1]))

0

In [70]:
kron([0im 1;1 0],[1 0;0 1])

4x4 Array{Complex{Int64},2}:
 0+0im  0+0im  1+0im  0+0im
 0+0im  0+0im  0+0im  1+0im
 1+0im  0+0im  0+0im  0+0im
 0+0im  1+0im  0+0im  0+0im

In [73]:
findfirst(FullCliffordCommands,FullCliffordCommands[4])

4

In [96]:
test=copy(FullCliffordCommands[104])

6-element Array{ASCIIString,1}:
 "setup(2)"    
 "hadamard(2)" 
 "hadamard(1)" 
 "cnot(2,1)"   
 "cnot(1,2)"   
 "output(svec)"

In [110]:
for i=1:size(FullCliffordCommands,1)
    if (size(FullCliffordCommands[i],1)==4)
        print(i,"\n")
        print(FullCliffordCommands[i])
    end
end

1928
ASCIIString["setup(2)","hadamard(2)","hadamard(1)","output(svec)"]1936
ASCIIString["setup(2)","hadamard(2)","hadamard(1)","output(svec)"]3640
ASCIIString["setup(2)","cnot(1,2)","cnot(2,1)","output(svec)"]3648
ASCIIString["setup(2)","cnot(1,2)","cnot(2,1)","output(svec)"]9176
ASCIIString["setup(2)","cnot(2,1)","cnot(1,2)","output(svec)"]9184
ASCIIString["setup(2)","cnot(2,1)","cnot(1,2)","output(svec)"]11505
ASCIIString["setup(2)","phase(1)","phase(1)","output(svec)"]11513
ASCIIString["setup(2)","phase(1)","phase(1)","output(svec)"]

In [113]:
11513-719*16


9